In [1]:
import numpy as np
import cv2
import h5py
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Conv2D,
    Concatenate,
    MaxPool2D,
    Dropout,
    Flatten,
)

2023-03-23 10:49:06.951439: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fizzer/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib
2023-03-23 10:49:06.951455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Loading all the data
imgs = np.load("/home/fizzer/ros_ws/cnn_trainer/data/imgs_0.npy")
vels = np.load("/home/fizzer/ros_ws/cnn_trainer/data/vels_0.npy")
print(imgs.shape)
print(vels.shape)

(40614, 90, 160, 1)
(40614, 2)


In [3]:
# remove all data where the robot is not moving forward
not_moving_fwd_idx = np.where(vels[:, 0] == 0.0)[0]
img_data = np.delete(imgs, not_moving_fwd_idx, axis=0)
vel_data = np.delete(vels, not_moving_fwd_idx, axis=0)

print(imgs.shape)
print(vels.shape)

(40614, 90, 160, 1)
(40614, 2)


In [4]:
print("starting data normalization")
img_data = img_data / 255
print("finished normalizing data")

starting data normalization
finished normalizing data


In [5]:
# shift
shift = 5

img_data = img_data[:-shift]
vel_data = vel_data[shift:]

In [6]:
straight = np.where(vel_data[:, 1] == 0.0)[0]
left = np.where(vel_data[:, 1] == 1.0)[0]
right = np.where(vel_data[:, 1] == -1.0)[0]

straight_img = img_data[straight]
left_img = img_data[left]
right_img = img_data[right]

straight_vel = vel_data[straight]
left_vel = vel_data[left]
right_vel = vel_data[right]

print(straight_img.shape)
print(left_img.shape)
print(right_img.shape)
print(straight_vel.shape)
print(left_vel.shape)
print(right_vel.shape)

(28141, 90, 160, 1)
(11246, 90, 160, 1)
(823, 90, 160, 1)
(28141, 2)
(11246, 2)
(823, 2)


In [18]:
straight_keep = straight[:len(straight)//2]
print(straight_keep.shape)
left_keep=left[:len(left)//2]
print(left_keep.shape)
right_keep=right
print(right_keep.shape)
all_keep = np.concatenate([straight_keep, left_keep, right_keep])
print(all_keep.shape)

(14070,)
(5623,)
(823,)
(20516,)


In [ ]:
img_data = img_data[all_keep]
vel_data=vel_data[all_keep]
print(img_data.shape)
print(vel_data.shape)

In [9]:
velocities_to_actions = {
    (0.25, 0.0): np.array([1, 0, 0]),
    (0.25, 1.0): np.array([0, 1, 0]),
    (0.25, -1.0): np.array([0, 0, 1]),
}

actions = np.array([velocities_to_actions[tuple(i)] for i in vel_data])

In [12]:
input_image = Input(shape=(90, 160, 1))
output_image = Conv2D(32, 4, padding="same")(input_image)
output_image = MaxPool2D(4)(output_image)

# output_image = Conv2D(32, 4, padding="same")(output_image)
# output_image = MaxPool2D(2)(output_image)
# output_image = Dropout(0.5)(output_image)

output_image = Conv2D(16, 4, padding="same")(output_image)
output_image = MaxPool2D(4)(output_image)
output_image = Dropout(0.5)(output_image)

# output_image = Conv2D(16, 4, padding="same")(output_image)
# output_image = MaxPool2D(2)(output_image)
# output_image = Dropout(0.5)(output_image)

output_image = Flatten()(output_image)

output_image = Dense(128, activation="relu")(output_image)
output_image = Dropout(0.5)(output_image)

output_image = Dense(64, activation="relu")(output_image)
output_image = Dropout(0.5)(output_image)

output_image = Dense(32, activation="relu")(output_image)
#output_image = Dropout(0.5)(output_image)

#output_image = Dense(16, activation="relu")(output_image)

output_image = Dense(3, activation="softmax")(output_image)


image_net = Model(input_image, output_image)
image_net.summary()
image_net.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 90, 160, 1)]      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 90, 160, 32)       544       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 22, 40, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 40, 16)        8208      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 10, 16)        0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 5, 10, 16)         0   

In [13]:
image_net.fit(img_data, actions, batch_size=128, epochs=2, validation_split=0.2)

2023-03-23 10:49:30.742214: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1852876800 exceeds 10% of free system memory.


Epoch 1/3
252/252 [==============================] - ETA: 0s - loss: 0.5119 - accuracy: 0.8143

2023-03-23 10:50:50.540794: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 463219200 exceeds 10% of free system memory.


252/252 [==============================] - 84s 330ms/step - loss: 0.5119 - accuracy: 0.8143 - val_loss: 0.3484 - val_accuracy: 0.8951
Epoch 2/3
252/252 [==============================] - 84s 332ms/step - loss: 0.3890 - accuracy: 0.8737 - val_loss: 0.3452 - val_accuracy: 0.8994
Epoch 3/3
252/252 [==============================] - 85s 336ms/step - loss: 0.3690 - accuracy: 0.8755 - val_loss: 0.3268 - val_accuracy: 0.8985


In [14]:
image_net.save('/home/fizzer/ros_ws/cnn_trainer/trained_model')

INFO:tensorflow:Assets written to: /home/fizzer/ros_ws/cnn_trainer/trained_model/assets


INFO:tensorflow:Assets written to: /home/fizzer/ros_ws/cnn_trainer/trained_model/assets


In [15]:
straight_actions = np.array([velocities_to_actions[tuple(i)] for i in straight_vel])
image_net.evaluate(straight_img, straight_actions)

2023-03-23 10:53:53.242807: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1620921600 exceeds 10% of free system memory.


880/880 [==============================] - 19s 22ms/step - loss: 0.2176 - accuracy: 0.9569


[0.21759997308254242, 0.9568601250648499]

In [16]:
left_actions = np.array([velocities_to_actions[tuple(i)] for i in left_vel])
image_net.evaluate(left_img, left_actions)

  7/352 [..............................] - ETA: 7s - loss: 0.4423 - accuracy: 0.7902

2023-03-23 10:54:21.981365: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 647769600 exceeds 10% of free system memory.


352/352 [==============================] - 7s 21ms/step - loss: 0.4880 - accuracy: 0.7757


[0.48796194791793823, 0.7756535410881042]

In [17]:
right_actions = np.array([velocities_to_actions[tuple(i)] for i in right_vel])
image_net.evaluate(right_img, right_actions)

26/26 [==============================] - 1s 21ms/step - loss: 2.2579 - accuracy: 0.0000e+00


[2.2579185962677, 0.0]